In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
# Wczytanie danych
data = pd.read_csv("diabetes.csv")
X = data.drop("Diabetic", axis=1)
y = data["Diabetic"]

# Podział danych na treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [3]:
# Pipeline dla regresji logistycznej (ze skalowaniem)
pipe_logistic = Pipeline([
    ('scaler', StandardScaler()),                    # Standaryzacja cech
    ('classifier', LogisticRegression(solver='liblinear'))  # Klasyfikator
])

# Pipeline dla lasu losowego (bez skalowania)
pipe_rf = Pipeline([
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Klasyfikator
])

In [4]:
# Trenowanie i porównanie modeli
for name, pipeline in [("Logistic Regression", pipe_logistic), ("Random Forest", pipe_rf)]:
    pipeline.fit(X_train, y_train)                             # Uczenie modelu
    y_pred = pipeline.predict(X_test)                          # Przewidywania klas
    y_proba = pipeline.predict_proba(X_test)[:, 1]             # Prawdopodobieństwo klasy pozytywnej

    print(f"\n{name}")
    print(classification_report(y_test, y_pred))               # Raport metryk
    print(f"AUC: {roc_auc_score(y_test, y_proba):.4f}")        # Metryka AUC


Logistic Regression
              precision    recall  f1-score   support

           0       0.81      0.88      0.84      3000
           1       0.71      0.59      0.65      1500

    accuracy                           0.78      4500
   macro avg       0.76      0.74      0.74      4500
weighted avg       0.78      0.78      0.78      4500

AUC: 0.8573

Random Forest
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3000
           1       0.91      0.88      0.90      1500

    accuracy                           0.93      4500
   macro avg       0.93      0.92      0.92      4500
weighted avg       0.93      0.93      0.93      4500

AUC: 0.9802


In [6]:
# GridSearch – strojenie hiperparametrów

# Pipeline do strojenia
pipe_grid = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(random_state=42))
])

# Siatka hiperparametrów do przeszukania
param_grid = {
    "rf__n_estimators": [50, 100, 150],         # liczba drzew
    "rf__max_depth": [3, 5, 10, None],          # maksymalna głębokość drzewa
    "rf__min_samples_split": [2, 5],            # min. próbki do podziału
    "rf__min_samples_leaf": [1, 2]              # min. próbki w liściu
}

# GridSearch z 5-krotną walidacją krzyżową
grid_search = GridSearchCV(pipe_grid, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Najlepsze hiperparametry
print("\nNajlepsze hiperparametry:")
print(grid_search.best_params_)

# Ewaluacja najlepszego modelu
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
y_proba_best = best_model.predict_proba(X_test)[:, 1]

print("\nWyniki dla modelu po GridSearch:")
print(classification_report(y_test, y_pred_best))
print(f"AUC (GridSearch): {roc_auc_score(y_test, y_proba_best):.4f}")


Najlepsze hiperparametry:
{'rf__max_depth': None, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 150}

Wyniki dla modelu po GridSearch:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3000
           1       0.91      0.88      0.90      1500

    accuracy                           0.93      4500
   macro avg       0.93      0.92      0.92      4500
weighted avg       0.93      0.93      0.93      4500

AUC (GridSearch): 0.9803
